In [1]:
# .env 파일에 OPENAI_API_KEY=<YOUR_KEY> 형식으로 API 키를 넣어 두었다고 가정합니다.
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수를 로드합니다.

import os
openai_api_key = os.getenv('OPENAI_API_KEY') 
# OpenAI API 키가 정상적으로 불러와졌는지 확인 (키 문자열 일부 출력)
print(openai_api_key[:8] + "****") if openai_api_key else print("API Key not found!")

sk-proj-****


In [2]:
from langchain_openai import ChatOpenAI
# OpenAI GPT-4 모델 초기화 (temperature 등의 파라미터 조정 가능)
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 템플릿 정의 (한국어 프롬프트)
product_prompt = PromptTemplate.from_template(
    "제품 이름: {product}\n" + 
    "이 제품의 특징과 장점을 매력적인 한 단락으로 설명해주세요."
)

# 2. 출력 파서 정의 (문자열 출력 파서)
output_parser = StrOutputParser()

# 3. LCEL 체인 구성: 프롬프트 → LLM → 출력파서
product_chain = product_prompt | llm | output_parser

# 4. 체인 실행 (.invoke) - 예시 입력으로 실행
example_input = {"product": "무선 블루투스 이어폰"}  # 예시로 "무선 블루투스 이어폰" 설명 생성
result = product_chain.invoke(example_input)

print(result)

이 무선 블루투스 이어폰은 깨끗한 사운드 품질과 함께 완전한 자유로움을 선사합니다. 끊김 없는 연결성과 오래 지속되는 배터리 수명으로 하루 종일 음악을 즐길 수 있습니다. 또한, 가벼운 무게와 부드러운 착용감으로 편안함을 제공하며, 스마트폰, 태블릿, 노트북 등 다양한 기기와 쉽게 페어링 할 수 있습니다. 이제 무선 블루투스 이어폰과 함께 어디서든 음악을 자유롭게 즐기세요.


In [5]:
from langchain_core.runnables import RunnablePassthrough

# 1. 이메일 제목 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    "다음 요청 내용을 바탕으로 이메일 제목을 지어주세요:\n{content}"
)

# 2. 이메일 본문 생성용 프롬프트 정의
body_prompt = PromptTemplate.from_template(
    "위에서 생성된 제목을 활용하여, 팀에게 보내는 정중한 이메일 본문을 작성해주세요.\n" + 
    "제목: {subject}\n본문:"
)

# 3. 두 프롬프트를 LLM과 결합한 체인 구성
email_chain = (
    subject_prompt          # 1단계: 제목 프롬프트 
    | llm                   # LLM 호출 -> 제목 생성
    | {"subject": RunnablePassthrough()}   # 출력된 제목을 subject 키로 매핑
    | body_prompt           # 2단계: 본문 프롬프트 (subject를 입력으로 받음)
    | llm                   # LLM 호출 -> 본문 생성
    | output_parser         # 최종 출력 파서 (본문 문자열로 변환)
)

# 4. 체인 실행 예시
email_request = {
    "content": "다음주에 프로젝트 진행 상황을 논의하기 위해 팀 회의를 요청드리는 이메일"
}
final_email = email_chain.invoke(email_request)
print(final_email)

안녕하세요, 팀 여러분,

다가오는 프로젝트에 대한 중요한 논의가 필요하여 이메일을 보내게 되었습니다. 우리 팀이 현재 진행 중인 프로젝트에 대한 상세한 논의를 위해 다음주에 회의를 개최하고자 합니다.

이번 회의에서는 각자의 작업 진행 상황을 공유하고, 향후 단계에 대한 계획을 세우는 시간을 가질 예정입니다. 이를 통해 우리 팀의 일정을 더욱 효과적으로 관리하고, 팀원 간의 협력을 강화하려고 합니다.

그리고, 이번 회의에는 모든 팀원의 참석이 필요하므로 본인의 일정을 확인하신 후 가능한 시간을 알려주시면 감사하겠습니다. 만약 참석이 어려운 경우, 미리 알려주시면 다른 시간으로 조정할 수 있도록 하겠습니다.

팀의 성공을 위해 각자의 노력이 필요함을 잊지 말아주시기 바랍니다. 여러분의 의견과 제안을 기다리고 있겠습니다.

감사합니다.

[본인의 이름]


In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableLambda

# 1. 요약 체인 정의 (프롬프트 → LLM)
summary_prompt = PromptTemplate.from_template("다음 글을 한 문단으로 요약해주세요:\n{text}")
summary_chain = summary_prompt | llm  # (StrOutputParser를 쓰지 않아도 기본 문자열 출력)

# 2. 이메일 체인은 이전 실습의 email_chain 활용 (이미 정의되었다고 가정; 없다면 위에서 정의한 코드 재사용)

# 3. 분기 조건 함수 정의 및 Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith("요약:")  # 입력이 "요약:"으로 시작하면 True

condition = RunnableLambda(is_summary_request)

# 4. 분기 체인 구성: (조건, 조건=True일 때 체인)와 False일 때 체인 지정
branch_chain = RunnableBranch(
    (condition, summary_chain),  # 조건이 True이면 summary_chain 실행
    # 람다 직접 구현해서 실행 가능
    # (lambda x : True if x.strip().startswith("요약:") else False , summary_chain),  # 조건이 True이면 summary_chain 실행
    email_chain                 # 조건 False이면 email_chain 실행
)

# 5. 다양한 입력에 대해 분기 체인 실행
input1 = "요약: 어제 진행된 회의에서는 프로젝트 일정 변경과 예산 조정에 대한 논의가 있었습니다."
result1 = branch_chain.invoke(input1)
print("[요약 요청 결과]\n", result1, "\n")

input2 = "이메일: 다음 주 월요일 프로젝트 회의 일정을 팀에 공지해줘."
result2 = branch_chain.invoke(input2)
print("[이메일 작성 요청 결과]\n", result2)

[요약 요청 결과]
 content='어제의 회의에서는 프로젝트 일정의 변경과 예산 조정에 대해 논의하였습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 64, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BatU4txz533gv9qucnmM3NJUZ5PcD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9bcea263-389a-4380-ba48-d6ddaf9bc329-0' usage_metadata={'input_tokens': 64, 'output_tokens': 33, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}} 

[이메일 작성 요청 결과]
 안녕하세요 팀원 여러분,

우선, 현재까지 프로젝트에 투입해주신 노력에 감사드립니다.

다음 주 월요일인 10월 11일 오전 10시에 프로젝트 회의를 진행할 예정입니다. 이번 회의에서는 우리 팀이 지금까지 추진해온 프로젝트의 진행 상황을 점검하고, 앞으로의 방향성을 함께 논의할 계획입니다.

회의에

In [8]:
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. 과거 대화 기록 생성
history = ChatMessageHistory()
history.add_user_message("내 이름은 홍길동이야.")
history.add_ai_message("홍길동님, 만나서 반가워요.")  # AI가 사용자 이름을 기억하여 응답

# 2. 채팅 프롬프트 템플릿 설정 (시스템 메시지 + 히스토리 + 사용자 메시지)
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("너는 친절하고 도움이 되는 어시스턴트야."),
    MessagesPlaceholder(variable_name="history"),  # 이전 대화 메모리 자리채움
    HumanMessagePromptTemplate.from_template("{input}")  # 새로운 사용자 입력
])

# 3. LCEL 체인 구성: 채팅 프롬프트 → LLM
chat_chain = chat_prompt | llm

# 4. 새로운 사용자 질문과 함께 체인 실행 (history와 input을 전달)
new_question = "내 이름이 뭐였지?"  # 사용자: 자기 이름을 물어봄
response = chat_chain.invoke({
    "history": history.messages,  # 이전 메시지 목록 전달
    "input": new_question
})

print(response.content)

당신의 이름은 홍길동이라고 말씀하셨어요.


In [9]:
base_prompt = PromptTemplate.from_template("오늘 날짜는 {date}입니다. {task}")
partial_prompt = base_prompt.partial(date="2025-05-23")  # date 변수 미리 채움
print(partial_prompt.format(task="해야 할 일을 알려줘."))

오늘 날짜는 2025-05-23입니다. 해야 할 일을 알려줘.


In [10]:
# .env 파일 로드
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일의 환경변수 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OWM_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")
# NEWS_API_KEY = os.getenv("NEWSAPI_API_KEY")

# 키 값 확인 (None이 아니라면 성공적으로 불러온 것입니다)
print("OpenAI Key present?", OPENAI_API_KEY is not None)
print("Weather API Key present?", OWM_API_KEY is not None)
# print("News API Key present?", NEWS_API_KEY is not None)

OpenAI Key present? True
Weather API Key present? True


In [51]:
import requests
from pydantic import BaseModel, Field

# 입력 스키마 정의
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름 (영문)")

def get_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 반환합니다."""
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",      # 섭씨 온도
        "lang": "kr"            # 한국어 응답 (설명이 한국어로 오도록)
    }
    response = requests.get(url, params=params)
    data = response.json()
    # API 응답에 따른 처리
    if data.get("cod") != 200:
        # 도시 정보를 찾지 못한 경우
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."
    # 필요한 정보 파싱
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    city_name = data["name"]  # 응답에 있는 도시 이름 (영문일 수 있음)
    return f"{city_name}의 현재 기온은 {temp}℃, 날씨는 {desc}입니다."

# get_weather_chain = RunnableLambda(lambda x: get_weather(x["city"]))
get_weather_chain = RunnableLambda(lambda x: get_weather(x.city))
# 체인 실행 예시
# weather_input = {"city": "Seoul"}  # 서울 날씨 조회 
weather_input = WeatherInput(city="Seoul") # 서울 날씨 조회 
weather_result = get_weather_chain.invoke(weather_input)
print(weather_result)  # 서울의 현재 날씨 출력


Seoul의 현재 기온은 19.76℃, 날씨는 맑음입니다.


In [ ]:
# 입력 스키마 정의
class NewsInput(BaseModel):
    keyword: str = Field(description="최신 뉴스를 검색할 키워드 (한글 또는 영문)")

def get_news(keyword: str) -> str:
    """주어진 키워드에 대한 최신 뉴스 제목을 반환합니다."""
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "language": "ko",         # 한국어 뉴스 기사 검색
        "sortBy": "publishedAt",  # 최신 뉴스 우선 정렬
        "pageSize": 1,            # 가장 최근 기사 1건만 가져오기
        "apiKey": api_key
    }
    res = requests.get(url, params=params)
    data = res.json()
    articles = data.get("articles")
    if not articles:
        return f"'{keyword}'에 대한 최신 뉴스가 없습니다."
    # 가장 첫 번째 뉴스를 선택
    top_article = articles[0]
    title = top_article.get("title", "(제목 없음)")
    source = top_article.get("source", {}).get("name", "")
    return f"'{keyword}' 관련 뉴스: {title} - {source}"

In [ ]:
#  공공기관 api
import requests
from datetime import datetime
import xmltodict
# .env 파일 로드
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일의 환경변수 로드
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")


def get_current_date():
    current_date = datetime.now().date()
    return current_date.strftime("%Y%m%d")

def get_current_hour():
    now = datetime.now()
    return datetime.now().strftime("%H%M")

int_to_weather = {
    "0": "맑음",
    "1": "비",
    "2": "비/눈",
    "3": "눈",
    "5": "빗방울",
    "6": "빗방울눈날림",
    "7": "눈날림"
}

def forecast(params):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'  # 초단기실황
    res = requests.get(url, params)

    # XML -> 딕셔너리
    xml_data = res.text
    dict_data = xmltodict.parse(xml_data)

    # 예외 처리: 데이터 없음
    response = dict_data.get("response", {})
    header = response.get("header", {})
    result_code = header.get("resultCode")

    if result_code != "00":
        # "00"이 아니면 오류 발생 (예: NO_DATA: 03)
        msg = header.get("resultMsg", "Unknown error")
        return f"[ERROR] 기상청 API 응답 오류: {msg}"

    body = response.get("body", {})
    items = body.get("items", {})
    item_list = items.get("item")

    # 예외 처리: item_list가 비어있거나 None인 경우
    if not item_list:
        return "[INFO] 기상청 API에 데이터가 없습니다."

    temp, sky = None, None
    for item in item_list:
        if item["category"] == "T1H":
            temp = item["obsrValue"]
        if item["category"] == "PTY":
            sky_code = item["obsrValue"]
            sky = int_to_weather.get(sky_code, "알 수 없음")

    if temp is None or sky is None:
        return "[INFO] 온도 또는 날씨 정보를 찾을 수 없습니다."

    return temp, sky

keys = WEATHER_API_KEY

params ={'serviceKey' : keys, 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'dataType' : 'XML', 
         'base_date' : get_current_date(), 
         'base_time' : get_current_hour(), 
         'nx' : '55', 
         'ny' : '127' }

forecast(params)


('18.7', '맑음')

In [57]:
from langchain_core.runnables import RunnableLambda

# 날씨 함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda input: get_weather(input["city"]))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구",
    args_schema=WeatherInput
)

# 뉴스 함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda input: get_news(input["keyword"]))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구",
    args_schema=NewsInput
)

print(f"Tool 이름: {weather_tool.name}, 설명: {weather_tool.description}")
print(f"Tool 이름: {news_tool.name}, 설명: {news_tool.description}")

Tool 이름: get_weather, 설명: 도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구
Tool 이름: get_news, 설명: 키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구


In [58]:
from langchain.agents import initialize_agent, AgentType

# 앞서 만든 Tool 객체 리스트 준비
tools = [weather_tool, news_tool]

# OpenAI 함수 기반 에이전트 생성
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True  # intermediate 단계 로그를 출력하여 확인
)

In [60]:
query2 = "서울의 날씨와 OpenAI에 대한 최신 뉴스 알려줘."
result2 = agent.run(query2)
print("Agent Result 2:", result2)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Seoul'}`


Seoul의 현재 기온은 19.76℃, 날씨는 맑음입니다.
Invoking: `get_news` with `{'keyword': 'OpenAI'}`


'OpenAI'에 대한 최신 뉴스가 없습니다.서울의 현재 기온은 19.76℃로, 날씨는 맑습니다. 그리고 'OpenAI'에 대한 최신 뉴스는 현재 없습니다. 다른 정보가 필요하시면 알려주세요.

> Finished chain.
Agent Result 2: 서울의 현재 기온은 19.76℃로, 날씨는 맑습니다. 그리고 'OpenAI'에 대한 최신 뉴스는 현재 없습니다. 다른 정보가 필요하시면 알려주세요.


In [54]:
print(get_news("OpenAI"))

'OpenAI'에 대한 최신 뉴스가 없습니다.


In [55]:
from langchain_core.runnables import RunnableLambda

# 날씨 함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda input: get_weather(input["city"]))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구",
    args_schema=WeatherInput
)

# 뉴스 함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda input: get_news(input["keyword"]))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구",
    args_schema=NewsInput
)

print(f"Tool 이름: {weather_tool.name}, 설명: {weather_tool.description}")
print(f"Tool 이름: {news_tool.name}, 설명: {news_tool.description}")

Tool 이름: get_weather, 설명: 도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구
Tool 이름: get_news, 설명: 키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구


In [56]:
user_question = "서울 날씨 알려줘."
# 1단계: Tool 함수 직접 호출
city_query = "Seoul"  # "서울"을 API에 맞게 영문 변환
weather_info = get_weather(city_query)
print("[툴 출력]", weather_info)

[툴 출력] Seoul의 현재 기온은 19.76℃, 날씨는 맑음입니다.


In [ ]:
query2 = "서울의 날씨와 OpenAI에 대한 최신 뉴스 알려줘."
result2 = agent.run(query2)
print("Agent Result 2:", result2)